In [1]:
import polars as pl

In [2]:
# use fully print tables 
pl.Config.set_tbl_cols(-1)
pl.Config.set_tbl_rows(-1)

polars.cfg.Config

In [3]:
df_receipt = pl.read_csv("ref_files/data/receipt.csv")
df_customer = pl.read_csv("ref_files/data/customer.csv")
df_store = pl.read_csv("ref_files/data/store.csv")
df_product = pl.read_csv("ref_files/data/product.csv")
df_category = pl.read_csv("ref_files/data/category.csv")

In [4]:
# 031
print(
    df_receipt.groupby("store_cd").agg([
        # 自由度: 0
        pl.col("amount").std(ddof=0)
    ]).sort("amount", reverse=True).head(5)
)

shape: (5, 2)
┌──────────┬────────────┐
│ store_cd ┆ amount     │
│ ---      ┆ ---        │
│ str      ┆ f64        │
╞══════════╪════════════╡
│ S13052   ┆ 663.391816 │
│ S14011   ┆ 553.456916 │
│ S14034   ┆ 544.903736 │
│ S13001   ┆ 543.536561 │
│ S13015   ┆ 543.409938 │
└──────────┴────────────┘


In [5]:
# 032
print(
    df_receipt.select([
        pl.col("amount").quantile(0).alias("q_0"),
        pl.col("amount").quantile(0.25).alias("q_25"),
        pl.col("amount").quantile(0.50).alias("q_50"),
        pl.col("amount").quantile(0.75).alias("q_75"),
        pl.col("amount").quantile(1.00).alias("q_100")
    ])
)

shape: (1, 5)
┌──────┬───────┬───────┬───────┬─────────┐
│ q_0  ┆ q_25  ┆ q_50  ┆ q_75  ┆ q_100   │
│ ---  ┆ ---   ┆ ---   ┆ ---   ┆ ---     │
│ f64  ┆ f64   ┆ f64   ┆ f64   ┆ f64     │
╞══════╪═══════╪═══════╪═══════╪═════════╡
│ 10.0 ┆ 102.0 ┆ 170.0 ┆ 288.0 ┆ 10925.0 │
└──────┴───────┴───────┴───────┴─────────┘


In [6]:
# 033
print(
    df_receipt.groupby("store_cd").agg([
        pl.col("amount").mean()
    ]).filter(
        pl.col("amount") >= 330
    ).sort("amount", reverse=True).head(10)
)

shape: (10, 2)
┌──────────┬────────────┐
│ store_cd ┆ amount     │
│ ---      ┆ ---        │
│ str      ┆ f64        │
╞══════════╪════════════╡
│ S13052   ┆ 402.86747  │
│ S13015   ┆ 351.11196  │
│ S13003   ┆ 350.915519 │
│ S14010   ┆ 348.791262 │
│ S13001   ┆ 348.470386 │
│ S13020   ┆ 337.879932 │
│ S14011   ┆ 335.718333 │
│ S14026   ┆ 332.340588 │
│ S13004   ┆ 330.943949 │
│ S13019   ┆ 330.208616 │
└──────────┴────────────┘


In [7]:
# 034

# Exprを変数化できるらしい
is_not_member_expr = pl.col("customer_id").str.starts_with("Z").is_not()

print(
    df_receipt.filter(
       is_not_member_expr
    ).groupby(
        "customer_id"
    ).agg([
        pl.col("amount").sum()
    ]).select([
        pl.col("amount").mean() 
    ])
)

shape: (1, 1)
┌─────────────┐
│ amount      │
│ ---         │
│ f64         │
╞═════════════╡
│ 2547.742235 │
└─────────────┘


In [8]:
# 035
print(
    df_receipt.filter(
       is_not_member_expr
    ).groupby(
        "customer_id"
    ).agg([
        pl.col("amount").sum()
    ]).with_columns(
        pl.col("amount").mean().alias("mean_amount")
    ).filter(
        pl.col("amount") >= pl.col("mean_amount")
    ).sort("customer_id").head(10)
)

shape: (10, 3)
┌────────────────┬────────┬─────────────┐
│ customer_id    ┆ amount ┆ mean_amount │
│ ---            ┆ ---    ┆ ---         │
│ str            ┆ i64    ┆ f64         │
╞════════════════╪════════╪═════════════╡
│ CS001115000010 ┆ 3044   ┆ 2547.742235 │
│ CS001205000006 ┆ 3337   ┆ 2547.742235 │
│ CS001214000009 ┆ 4685   ┆ 2547.742235 │
│ CS001214000017 ┆ 4132   ┆ 2547.742235 │
│ CS001214000052 ┆ 5639   ┆ 2547.742235 │
│ CS001215000040 ┆ 3496   ┆ 2547.742235 │
│ CS001304000006 ┆ 3726   ┆ 2547.742235 │
│ CS001305000005 ┆ 3485   ┆ 2547.742235 │
│ CS001305000011 ┆ 4370   ┆ 2547.742235 │
│ CS001315000180 ┆ 3300   ┆ 2547.742235 │
└────────────────┴────────┴─────────────┘


In [9]:
# 036
print(
    df_receipt.join(
        df_store.select(["store_cd", "store_name"]), 
        how = "inner", 
        on = "store_cd"
    ).head(10)
)

shape: (10, 10)
┌─────────┬───────────┬────────┬──────────┬────────────┬───────────┬──────────┬────────┬────────┬──────────┐
│ sales_y ┆ sales_epo ┆ store_ ┆ receipt_ ┆ receipt_su ┆ customer_ ┆ product_ ┆ quanti ┆ amount ┆ store_na │
│ md      ┆ ch        ┆ cd     ┆ no       ┆ b_no       ┆ id        ┆ cd       ┆ ty     ┆ ---    ┆ me       │
│ ---     ┆ ---       ┆ ---    ┆ ---      ┆ ---        ┆ ---       ┆ ---      ┆ ---    ┆ i64    ┆ ---      │
│ i64     ┆ i64       ┆ str    ┆ i64      ┆ i64        ┆ str       ┆ str      ┆ i64    ┆        ┆ str      │
╞═════════╪═══════════╪════════╪══════════╪════════════╪═══════════╪══════════╪════════╪════════╪══════════╡
│ 2018110 ┆ 154120320 ┆ S14006 ┆ 112      ┆ 1          ┆ CS0062140 ┆ P0703050 ┆ 1      ┆ 158    ┆ 葛が谷店 │
│ 3       ┆ 0         ┆        ┆          ┆            ┆ 00001     ┆ 12       ┆        ┆        ┆          │
│ 2018111 ┆ 154249920 ┆ S13008 ┆ 1132     ┆ 2          ┆ CS0084150 ┆ P0707010 ┆ 1      ┆ 81     ┆ 成城店   │
│ 8       

In [10]:
# 037
print(
    df_product.join(
        df_category.select(["category_small_cd", "category_small_name"]), 
        how = "inner", 
        on = "category_small_cd"
    ).head(10)
)

shape: (10, 7)
┌────────────┬───────────────┬──────────────┬──────────────┬────────────┬───────────┬──────────────┐
│ product_cd ┆ category_majo ┆ category_med ┆ category_sma ┆ unit_price ┆ unit_cost ┆ category_sma │
│ ---        ┆ r_cd          ┆ ium_cd       ┆ ll_cd        ┆ ---        ┆ ---       ┆ ll_name      │
│ str        ┆ ---           ┆ ---          ┆ ---          ┆ i64        ┆ i64       ┆ ---          │
│            ┆ i64           ┆ i64          ┆ i64          ┆            ┆           ┆ str          │
╞════════════╪═══════════════╪══════════════╪══════════════╪════════════╪═══════════╪══════════════╡
│ P040101001 ┆ 4             ┆ 401          ┆ 40101        ┆ 198        ┆ 149       ┆ 弁当類       │
│ P040101002 ┆ 4             ┆ 401          ┆ 40101        ┆ 218        ┆ 164       ┆ 弁当類       │
│ P040101003 ┆ 4             ┆ 401          ┆ 40101        ┆ 230        ┆ 173       ┆ 弁当類       │
│ P040101004 ┆ 4             ┆ 401          ┆ 40101        ┆ 248        ┆ 186       ┆

In [40]:
# 038
print(
    df_customer.filter(
        (pl.col("gender_cd") == 1) &
        (is_not_member_expr)
    ).join(
        df_receipt.select(["customer_id", "amount"]), 
        how = "left", 
        on = "customer_id"
    )
    .groupby("customer_id")
    .agg([
        pl.col("amount").sum().fill_null(0)
    ]).head(10)
)

shape: (10, 2)
┌────────────────┬────────┐
│ customer_id    ┆ amount │
│ ---            ┆ ---    │
│ str            ┆ i64    │
╞════════════════╪════════╡
│ CS015213000056 ┆ 0      │
│ CS002712000013 ┆ 0      │
│ CS028314000011 ┆ 188    │
│ CS024411000003 ┆ 398    │
│ CS023412000201 ┆ 0      │
│ CS002512000420 ┆ 0      │
│ CS005414000090 ┆ 5301   │
│ CS011815000031 ┆ 0      │
│ CS004414000089 ┆ 3557   │
│ CS006515000237 ┆ 0      │
└────────────────┴────────┘


In [47]:
# 039
df_a = df_receipt.filter((is_not_member_expr)).groupby("customer_id").agg(
        pl.col("sales_ymd").n_unique()
    ).sort(
        "sales_ymd", reverse=True
    ).head(20)

df_b = df_receipt.filter((is_not_member_expr)).groupby("customer_id").agg(
        pl.col("amount").sum()
    ).sort(
        "amount", reverse=True
    ).head(20)

print(
    df_a.join(
        df_b,
        on = "customer_id",
        how = "outer"
    )
)

shape: (34, 3)
┌────────────────┬───────────┬────────┐
│ customer_id    ┆ sales_ymd ┆ amount │
│ ---            ┆ ---       ┆ ---    │
│ str            ┆ u32       ┆ i64    │
╞════════════════╪═══════════╪════════╡
│ CS017415000097 ┆ 20        ┆ 23086  │
│ CS015415000185 ┆ 22        ┆ 20153  │
│ CS031414000051 ┆ 19        ┆ 19202  │
│ CS028415000007 ┆ 21        ┆ 19127  │
│ CS001605000009 ┆ null      ┆ 18925  │
│ CS010214000010 ┆ 22        ┆ 18585  │
│ CS016415000141 ┆ 20        ┆ 18372  │
│ CS006515000023 ┆ null      ┆ 18372  │
│ CS011414000106 ┆ null      ┆ 18338  │
│ CS038415000104 ┆ null      ┆ 17847  │
│ CS035414000024 ┆ null      ┆ 17615  │
│ CS021515000089 ┆ null      ┆ 17580  │
│ CS032414000072 ┆ null      ┆ 16563  │
│ CS016415000101 ┆ null      ┆ 16348  │
│ CS011415000006 ┆ null      ┆ 16094  │
│ CS034415000047 ┆ null      ┆ 16083  │
│ CS007514000094 ┆ null      ┆ 15735  │
│ CS009414000059 ┆ null      ┆ 15492  │
│ CS030415000034 ┆ null      ┆ 15468  │
│ CS015515000034 ┆ null  

In [55]:
# 040
(
    df_store.with_column(pl.lit(0).alias("key")).join(
        df_product.with_column(pl.lit(0).alias("key")),
        on = "key",
        how = "outer"
    )
).shape

(531590, 17)